Segmenting and Clustering Neighborhoods in Toronto

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto based on the postalcode and borough information.. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas  dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

Your submission will be a link to your Jupyter Notebook on your Github repository.

Part 1. Import the data from Wiki to the pandas dataframe

In [28]:
!pip install beautifulsoup4
!pip install geopy

     |████████████████████████████████| 122kB 15.9MB/s eta 0:00:01


In [39]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import matplotlib.colors as colors
import matplotlib.cm as cm
import folium # map rendering library
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import requests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans

Scrape the List of postal codes of Canada

In [35]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html.parser")
#table_contents = pd.DataFrame(columns = ['PostalCode', 'Borough','Neighborhood'])

for row in soup.find_all("tbody")[1].find_all("tr"):
    page = requests.get(url)
    soup =  BeautifulSoup(page.content,"html.parser")
    table_contents=[]
    table=soup.find('table')
    for row in table.findAll('td'):
        cell = {}
        if row.span.text=='Not assigned':
            pass
        else:
            cell['PostalCode'] = row.p.text[:3]
            cell['Borough'] = (row.span.text).split('(')[0]
            cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
            table_contents.append(cell)

In [36]:
df=pd.DataFrame(table_contents)

In [37]:
df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [38]:
df.shape

(103, 3)

Part 2. Add latitude and longitude

In [41]:
df_coordinates = pd.read_csv(r"Geospatial_Coordinates.csv").set_index("Postal Code")
df_coordinates.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [42]:
df_coordinates = df_coordinates.to_dict()

In [43]:
df["Latitude"] = df["PostalCode"].apply(lambda postal_code:df_coordinates['Latitude'][postal_code])
df["Longitude"] = df["PostalCode"].apply(lambda postal_code:df_coordinates['Longitude'][postal_code])
df.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
